In [1]:
# ATSIT 2019: dimensional speech emotion recognition from text feature

# uncomment these to run on CPU only
import os
#os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
#os.environ["CUDA_VISIBLE_DEVICES"]=""

import numpy as np
import matplotlib.pyplot as plt
import pickle
import keras.backend as K
import pandas as pd
from keras.models import Model, Sequential
from keras.layers import Input, Dense, Masking, LSTM, TimeDistributed, Bidirectional, Embedding,GRU, Dropout, Flatten, concatenate, Convolution1D, BatchNormalization
from keras.utils import to_categorical
from sklearn.preprocessing import MinMaxScaler
from keras.layers.convolutional import Conv2D
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence
from keras.layers.core import Dense, Activation
from keras.callbacks import EarlyStopping
import codecs
from keras.optimizers import SGD, Adam, RMSprop
import io
import random as rn
import tensorflow as tf
from keras.layers.core import Dense, Activation
rn.seed(123)
np.random.seed(99)
tf.set_random_seed(1234)
# load feature and labels
feat = np.load('../../dimensionalSM/data/feat_34_hfs.npy')
vad = np.load('../../dimensionalSM/data/y_egemaps.npy')
# remove outlier, < 1, > 5
vad = np.where(vad==5.5, 5.0, vad)
vad = np.where(vad==0.5, 1.0, vad)

# standardization
scaled_feature = False
if scaled_feature == True:
    scaler = StandardScaler()
    scaler = scaler.fit(feat.reshape(feat.shape[0]*feat.shape[1], feat.shape[2]))
    scaled_feat = scaler.transform(feat.reshape(feat.shape[0]*feat.shape[1], feat.shape[2]))
    scaled_feat = scaled_feat.reshape(feat.shape[0], feat.shape[1], feat.shape[2])
    feat = scaled_feat
else:
    feat = feat

scaled_vad = True

# standardization
if scaled_vad:
    scaler = MinMaxScaler(feature_range=(-1, 1))
    scaler = scaler.fit(vad) #.reshape(vad.shape[0]*vad.shape[1], vad.shape[2]))
    scaled_vad = scaler.transform(vad) #.reshape(vad.shape[0]*vad.shape[1], vad.shape[2]))
    vad = scaled_vad 
else:
    vad = vad

# Concordance correlation coefficient (CCC)-based loss function - using non-inductive statistics
def ccc(gold, pred):
    gold       = K.squeeze(gold, axis=-1)
    pred       = K.squeeze(pred, axis=-1)
    gold_mean  = K.mean(gold, axis=-1, keepdims=True)
    pred_mean  = K.mean(pred, axis=-1, keepdims=True)
    covariance = (gold-gold_mean)*(pred-pred_mean)
    gold_var   = K.mean(K.square(gold-gold_mean), axis=-1, keepdims=True)
    pred_var   = K.mean(K.square(pred-pred_mean), axis=-1, keepdims=True)
    ccc        = K.constant(2.) * covariance / (gold_var + pred_var + K.square(gold_mean - pred_mean) + K.common.epsilon())
    return ccc


def ccc_loss(gold, pred):  
    # input (num_batches, seq_len, 1)
    ccc_loss   = K.constant(1.) - ccc(gold, pred)
    return ccc_loss
import pickle
with open('../../dimensionalSM/IEMOCAP_full_release/data_collected_333.pickle', 'rb') as handle:
    data2 = pickle.load(handle)
    
x_train_mocap = []
counter = 0
for ses_mod in data2:
    x_head = ses_mod['mocap_head']
    if(x_head.shape != (200,18)):
        x_head = np.zeros((200,18))   
    x_head[np.isnan(x_head)]=0
    x_hand = ses_mod['mocap_hand']
    if(x_hand.shape != (200,6)):
        x_hand = np.zeros((200,6))   
    x_hand[np.isnan(x_hand)]=0
    x_rot = ses_mod['mocap_rot']
    if(x_rot.shape != (200,165)):
        x_rot = np.zeros((200,165))   
    x_rot[np.isnan(x_rot)]=0
    x_mocap = np.concatenate((x_head, x_hand), axis=1)
    x_mocap = np.concatenate((x_mocap, x_rot), axis=1)
    x_train_mocap.append( x_mocap )
    
x_train_mocap = np.array(x_train_mocap)
x_train_mocap = x_train_mocap.reshape(-1,200,189,1)
x_train_mocap.shape
# remove outlier
# outlier = np.array([1674, 3427, 5086, 5093, 5096, 5104, 7821])
# mask = np.ones(len(vad), np.bool)
# mask[outlier] = 0
# vad = vad[mask]
# x_train_mocap = x_train_mocap[mask]


Using TensorFlow backend.


(10039, 200, 189, 1)

In [2]:
# split train/test
split = 7869

In [3]:
# API model, if use RNN, first two rnn layer must return_sequences=True
def api_model(a,b,c):
    
    input_mocap = Input(shape=(200,189,1))
    model_mocap0 = BatchNormalization()(input_mocap)
    model_mocap1 = Conv2D(64, 3, strides=(3, 3), padding="same")(model_mocap0)
    model_mocap2 = Dropout(0.2)(model_mocap1)
    model_mocap3 = Activation('relu')(model_mocap2)
    model_mocap4 = Conv2D(128, 3,strides=(3, 3), padding="same")(model_mocap3)
    model_mocap5 = Dropout(0.2)(model_mocap4)
    model_mocap6 = Activation('relu')(model_mocap5)
    model_mocap7 = Conv2D(256, 3,strides=(3, 3), padding="same")(model_mocap6)
    model_mocap8 = Dropout(0.2)(model_mocap7) 
    model_mocap9 = Activation('relu')(model_mocap8)
    model_mocap10 = Conv2D(128, 3,strides=(3, 3), padding="same")(model_mocap9)   
    model_mocap11 = Dropout(0.2)(model_mocap10)  
    model_mocap12 = Activation('relu')(model_mocap11)
    model_mocap13 = Conv2D(64, 3,strides=(3, 3), padding="same")(model_mocap12)  
    model_mocap14 = Dropout(0.2)(model_mocap13) 
    model_mocap15 = Activation('relu')(model_mocap14)
    model_mocap = Flatten()(model_mocap15) 
    #model_mocap = Dropout(dropout_rate)(model_mocap16)

    target_names = ('v', 'a', 'd')
    model_combined = [Dense(1, name=name)(model_mocap) for name in target_names]

    model = Model(input_mocap, model_combined) 

    model.compile(loss=ccc_loss, 
                  loss_weights={'v': a, 'a': b, 'd': c},
                  optimizer='rmsprop', metrics=[ccc])
    return model

#def main(alpha, beta, gamma):
# model = api_model(1, 1, 1)
#odel = api_model(0.1, 0.5, 0.4)
#model.summary()
# main function for for LOSO


In [4]:
# #speech_to_npy
# model1 = api_model(1,1,1)
# earlystop = EarlyStopping(monitor='val_loss', mode='min', patience=10,
#                               restore_best_weights=True)
# hist1 = model1.fit(x_train_mocap[:split], vad[:split].T.tolist(), epochs=50, 
#                    batch_size=32, verbose=1, validation_split=0.2, 
#                    callbacks=[earlystop])

# eval_metrik1 = model1.evaluate(x_train_mocap[split:], vad[split:].T.tolist())
# print(eval_metrik1)

# # make prediction
# predict = model1.predict(x_train_mocap[6290:], batch_size=32)
# np.save('mocap_batch_npy7869', 
#          np.array(predict).reshape(3, 3749).T)


In [5]:
# # 7TEST1
# earlystop = EarlyStopping(monitor='val_loss', mode='min', patience=10,
#                           restore_best_weights=True)
# hist = model.fit(x_train_mocap[:split], vad[:split].T.tolist(), batch_size=32, #best:8
#                   validation_split=0.2, epochs=50, verbose=1, shuffle=True, 
#                   callbacks=[earlystop])
# metrik = model.evaluate(x_train_mocap[split:], vad[split:].T.tolist())
# print(metrik[-3:])

In [6]:
def main(time):
    model = api_model(1,1,1)
    earlystop = EarlyStopping(monitor='val_loss', mode='min', patience=10,
                              restore_best_weights=True)
    hist = model.fit( x_train_mocap[:split], 
                      vad[:split].T.tolist(), batch_size=32, #best:8
                      validation_split=0.2, epochs=50, verbose=1, shuffle=True, callbacks=[earlystop]
                      )
    metrik = model.evaluate( x_train_mocap[split:], vad[split:].T.tolist())
    print(metrik[-3:])
    # make prediction
    predict = model.predict(x_train_mocap[6290:], batch_size=32)
    np.save('npy7869/mocap_npy-7869batch'+str(time)+'.npy',  
             np.array(predict).reshape(3, 3749).T)
    return metrik
    
if __name__ == '__main__':
    #loss_weight_step = np.around(np.arange(0.0, 1.1, 0.1), decimals=1)
    #dropout_rates = np.arange(0.1, 0.5, 0.1)
    weight = []
    #for i in dropout_rates:
    for time in range(20):
        best = main(time)
        weight.append([time, best[-3], best[-2], best[-1], np.mean(best[-3:])])
        print(time)
    weight_df = pd.DataFrame(weight)
    weight_df.to_csv('npy7869/mocap_2DCNN7869-batch-1.csv', header=['time','v','a','d', 'ave'])


Instructions for updating:
If using Keras pass *_constraint arguments to layers.

Train on 6295 samples, validate on 1574 samples
Epoch 1/50
6295/6295 [==============================] - 49s 8ms/step - loss: 2.6527 - v_loss: 0.8318 - a_loss: 0.9133 - d_loss: 0.9077 - v_ccc: 0.1682 - a_ccc: 0.0867 - d_ccc: 0.0923 - val_loss: 2.6143 - val_v_loss: 0.8947 - val_a_loss: 0.8450 - val_d_loss: 0.8698 - val_v_ccc: 0.0954 - val_a_ccc: 0.1587 - val_d_ccc: 0.1317
Epoch 2/50
6295/6295 [==============================] - 45s 7ms/step - loss: 2.0588 - v_loss: 0.5198 - a_loss: 0.7575 - d_loss: 0.7812 - v_ccc: 0.4801 - a_ccc: 0.2424 - d_ccc: 0.2187 - val_loss: 2.5943 - val_v_loss: 0.8538 - val_a_loss: 0.8485 - val_d_loss: 0.8880 - val_v_ccc: 0.1377 - val_a_ccc: 0.1554 - val_d_ccc: 0.1127
Epoch 3/50
6295/6295 [==============================] - 44s 7ms/step - loss: 1.8892 - v_loss: 0.4729 - a_loss: 0.7067 - d_loss: 0.7091 - v_ccc: 0.5269 - a_ccc: 0.2932 - d_ccc: 0.2908 - val_loss: 2.4900 - val_v_loss: 0.83

6295/6295 [==============================] - 45s 7ms/step - loss: 1.2646 - v_loss: 0.3286 - a_loss: 0.4733 - d_loss: 0.4632 - v_ccc: 0.6715 - a_ccc: 0.5268 - d_ccc: 0.5371 - val_loss: 2.3248 - val_v_loss: 0.7873 - val_a_loss: 0.7405 - val_d_loss: 0.8082 - val_v_ccc: 0.2116 - val_a_ccc: 0.2660 - val_d_ccc: 0.1975
Epoch 25/50
6295/6295 [==============================] - 46s 7ms/step - loss: 1.2551 - v_loss: 0.3240 - a_loss: 0.4664 - d_loss: 0.4645 - v_ccc: 0.6758 - a_ccc: 0.5338 - d_ccc: 0.5353 - val_loss: 2.2756 - val_v_loss: 0.7815 - val_a_loss: 0.7231 - val_d_loss: 0.7828 - val_v_ccc: 0.2177 - val_a_ccc: 0.2835 - val_d_ccc: 0.2232
Epoch 26/50
6295/6295 [==============================] - 45s 7ms/step - loss: 1.2471 - v_loss: 0.3205 - a_loss: 0.4648 - d_loss: 0.4625 - v_ccc: 0.6796 - a_ccc: 0.5360 - d_ccc: 0.5374 - val_loss: 2.3773 - val_v_loss: 0.8108 - val_a_loss: 0.7612 - val_d_loss: 0.8149 - val_v_ccc: 0.1883 - val_a_ccc: 0.2444 - val_d_ccc: 0.1900
Epoch 27/50
6295/6295 [===========

Epoch 21/50
6295/6295 [==============================] - 44s 7ms/step - loss: 1.3073 - v_loss: 0.3353 - a_loss: 0.4832 - d_loss: 0.4883 - v_ccc: 0.6644 - a_ccc: 0.5168 - d_ccc: 0.5116 - val_loss: 2.2929 - val_v_loss: 0.7909 - val_a_loss: 0.7552 - val_d_loss: 0.7564 - val_v_ccc: 0.2078 - val_a_ccc: 0.2503 - val_d_ccc: 0.2489
Epoch 22/50
6295/6295 [==============================] - 44s 7ms/step - loss: 1.2922 - v_loss: 0.3285 - a_loss: 0.4766 - d_loss: 0.4874 - v_ccc: 0.6716 - a_ccc: 0.5238 - d_ccc: 0.5125 - val_loss: 2.2065 - val_v_loss: 0.7533 - val_a_loss: 0.7213 - val_d_loss: 0.7439 - val_v_ccc: 0.2458 - val_a_ccc: 0.2848 - val_d_ccc: 0.2629
Epoch 23/50
6295/6295 [==============================] - 44s 7ms/step - loss: 1.2845 - v_loss: 0.3319 - a_loss: 0.4748 - d_loss: 0.4782 - v_ccc: 0.6679 - a_ccc: 0.5251 - d_ccc: 0.5224 - val_loss: 2.2176 - val_v_loss: 0.7634 - val_a_loss: 0.7243 - val_d_loss: 0.7383 - val_v_ccc: 0.2317 - val_a_ccc: 0.2821 - val_d_ccc: 0.2686
Epoch 24/50
6295/6295 

Epoch 21/50
6295/6295 [==============================] - 44s 7ms/step - loss: 1.2961 - v_loss: 0.3384 - a_loss: 0.4802 - d_loss: 0.4785 - v_ccc: 0.6618 - a_ccc: 0.5200 - d_ccc: 0.5220 - val_loss: 2.2041 - val_v_loss: 0.7728 - val_a_loss: 0.7060 - val_d_loss: 0.7304 - val_v_ccc: 0.2199 - val_a_ccc: 0.3004 - val_d_ccc: 0.2756
Epoch 22/50
6295/6295 [==============================] - 43s 7ms/step - loss: 1.2793 - v_loss: 0.3325 - a_loss: 0.4698 - d_loss: 0.4766 - v_ccc: 0.6673 - a_ccc: 0.5302 - d_ccc: 0.5231 - val_loss: 2.2451 - val_v_loss: 0.7892 - val_a_loss: 0.7027 - val_d_loss: 0.7649 - val_v_ccc: 0.2076 - val_a_ccc: 0.3055 - val_d_ccc: 0.2418
Epoch 23/50
6295/6295 [==============================] - 44s 7ms/step - loss: 1.2608 - v_loss: 0.3228 - a_loss: 0.4716 - d_loss: 0.4668 - v_ccc: 0.6772 - a_ccc: 0.5282 - d_ccc: 0.5338 - val_loss: 2.2756 - val_v_loss: 0.7893 - val_a_loss: 0.7452 - val_d_loss: 0.7440 - val_v_ccc: 0.2025 - val_a_ccc: 0.2604 - val_d_ccc: 0.2614
Epoch 24/50
6295/6295 

6295/6295 [==============================] - 44s 7ms/step - loss: 1.0044 - v_loss: 0.2765 - a_loss: 0.3608 - d_loss: 0.3672 - v_ccc: 0.7234 - a_ccc: 0.6390 - d_ccc: 0.6332 - val_loss: 2.2246 - val_v_loss: 0.7347 - val_a_loss: 0.7121 - val_d_loss: 0.7867 - val_v_ccc: 0.2596 - val_a_ccc: 0.2946 - val_d_ccc: 0.2212
Epoch 47/50
6295/6295 [==============================] - 44s 7ms/step - loss: 0.9941 - v_loss: 0.2749 - a_loss: 0.3591 - d_loss: 0.3604 - v_ccc: 0.7251 - a_ccc: 0.6408 - d_ccc: 0.6399 - val_loss: 2.2380 - val_v_loss: 0.7538 - val_a_loss: 0.7213 - val_d_loss: 0.7700 - val_v_ccc: 0.2417 - val_a_ccc: 0.2848 - val_d_ccc: 0.2355
Epoch 48/50
6295/6295 [==============================] - 44s 7ms/step - loss: 0.9922 - v_loss: 0.2655 - a_loss: 0.3620 - d_loss: 0.3647 - v_ccc: 0.7345 - a_ccc: 0.6380 - d_ccc: 0.6354 - val_loss: 2.2652 - val_v_loss: 0.7541 - val_a_loss: 0.7248 - val_d_loss: 0.7933 - val_v_ccc: 0.2416 - val_a_ccc: 0.2810 - val_d_ccc: 0.2121
Epoch 49/50
6295/6295 [===========

...

6295/6295 [==============================] - 42s 7ms/step - loss: 1.2080 - v_loss: 0.3151 - a_loss: 0.4432 - d_loss: 0.4496 - v_ccc: 0.6849 - a_ccc: 0.5569 - d_ccc: 0.5502 - val_loss: 2.2451 - val_v_loss: 0.7487 - val_a_loss: 0.7035 - val_d_loss: 0.8031 - val_v_ccc: 0.2499 - val_a_ccc: 0.3030 - val_d_ccc: 0.2020
Epoch 27/50
6295/6295 [==============================] - 43s 7ms/step - loss: 1.1908 - v_loss: 0.3083 - a_loss: 0.4431 - d_loss: 0.4394 - v_ccc: 0.6916 - a_ccc: 0.5569 - d_ccc: 0.5608 - val_loss: 2.2210 - val_v_loss: 0.7402 - val_a_loss: 0.6965 - val_d_loss: 0.7933 - val_v_ccc: 0.2565 - val_a_ccc: 0.3100 - val_d_ccc: 0.2125
Epoch 28/50
2170/2170 [==============================] - 2s 1ms/step
[0.2591266334056854, 0.28316593170166016, 0.2518126368522644]
16
Train on 6295 samples, validate on 1574 samples
Epoch 1/50
6295/6295 [==============================] - 45s 7ms/step - loss: 2.7794 - v_loss: 0.8703 - a_loss: 0.9502 - d_loss: 0.9580 - v_ccc: 0.1293 - a_ccc: 0.0494 - d_ccc: 0.

6295/6295 [==============================] - 42s 7ms/step - loss: 1.2536 - v_loss: 0.3226 - a_loss: 0.4689 - d_loss: 0.4631 - v_ccc: 0.6773 - a_ccc: 0.5315 - d_ccc: 0.5376 - val_loss: 2.2943 - val_v_loss: 0.8217 - val_a_loss: 0.7148 - val_d_loss: 0.7642 - val_v_ccc: 0.1718 - val_a_ccc: 0.2924 - val_d_ccc: 0.2416
Epoch 23/50
6295/6295 [==============================] - 42s 7ms/step - loss: 1.2301 - v_loss: 0.3137 - a_loss: 0.4631 - d_loss: 0.4533 - v_ccc: 0.6862 - a_ccc: 0.5371 - d_ccc: 0.5466 - val_loss: 2.3095 - val_v_loss: 0.8042 - val_a_loss: 0.7357 - val_d_loss: 0.7801 - val_v_ccc: 0.1947 - val_a_ccc: 0.2699 - val_d_ccc: 0.2258
Epoch 24/50
6295/6295 [==============================] - 43s 7ms/step - loss: 1.2069 - v_loss: 0.3176 - a_loss: 0.4403 - d_loss: 0.4497 - v_ccc: 0.6823 - a_ccc: 0.5600 - d_ccc: 0.5508 - val_loss: 2.2874 - val_v_loss: 0.8135 - val_a_loss: 0.7186 - val_d_loss: 0.7646 - val_v_ccc: 0.1827 - val_a_ccc: 0.2879 - val_d_ccc: 0.2421
Epoch 25/50
6295/6295 [===========

Epoch 16/50
6295/6295 [==============================] - 43s 7ms/step - loss: 1.4161 - v_loss: 0.3602 - a_loss: 0.5293 - d_loss: 0.5271 - v_ccc: 0.6401 - a_ccc: 0.4708 - d_ccc: 0.4730 - val_loss: 2.2844 - val_v_loss: 0.7696 - val_a_loss: 0.7472 - val_d_loss: 0.7744 - val_v_ccc: 0.2266 - val_a_ccc: 0.2587 - val_d_ccc: 0.2303
Epoch 17/50
6295/6295 [==============================] - 42s 7ms/step - loss: 1.3963 - v_loss: 0.3483 - a_loss: 0.5223 - d_loss: 0.5258 - v_ccc: 0.6517 - a_ccc: 0.4776 - d_ccc: 0.4744 - val_loss: 2.3338 - val_v_loss: 0.8270 - val_a_loss: 0.7473 - val_d_loss: 0.7636 - val_v_ccc: 0.1675 - val_a_ccc: 0.2578 - val_d_ccc: 0.2409
Epoch 18/50
6295/6295 [==============================] - 42s 7ms/step - loss: 1.3961 - v_loss: 0.3523 - a_loss: 0.5265 - d_loss: 0.5171 - v_ccc: 0.6476 - a_ccc: 0.4735 - d_ccc: 0.4828 - val_loss: 2.3412 - val_v_loss: 0.7822 - val_a_loss: 0.7427 - val_d_loss: 0.8247 - val_v_ccc: 0.2163 - val_a_ccc: 0.2629 - val_d_ccc: 0.1796
Epoch 19/50
6295/6295 

6295/6295 [==============================] - 43s 7ms/step - loss: 1.1130 - v_loss: 0.2938 - a_loss: 0.4093 - d_loss: 0.4096 - v_ccc: 0.7062 - a_ccc: 0.5905 - d_ccc: 0.5902 - val_loss: 2.2719 - val_v_loss: 0.7953 - val_a_loss: 0.7192 - val_d_loss: 0.7664 - val_v_ccc: 0.2039 - val_a_ccc: 0.2860 - val_d_ccc: 0.2383
Epoch 42/50
6295/6295 [==============================] - 42s 7ms/step - loss: 1.0897 - v_loss: 0.2873 - a_loss: 0.4006 - d_loss: 0.4014 - v_ccc: 0.7126 - a_ccc: 0.5993 - d_ccc: 0.5984 - val_loss: 2.2856 - val_v_loss: 0.7624 - val_a_loss: 0.7193 - val_d_loss: 0.8141 - val_v_ccc: 0.2352 - val_a_ccc: 0.2878 - val_d_ccc: 0.1914
Epoch 43/50
2170/2170 [==============================] - 2s 1ms/step
[0.29303452372550964, 0.2651849091053009, 0.25471487641334534]
18
Train on 6295 samples, validate on 1574 samples
Epoch 1/50
6295/6295 [==============================] - 45s 7ms/step - loss: 2.7080 - v_loss: 0.8385 - a_loss: 0.9206 - d_loss: 0.9476 - v_ccc: 0.1606 - a_ccc: 0.0792 - d_ccc: 0

6295/6295 [==============================] - 43s 7ms/step - loss: 1.2551 - v_loss: 0.3246 - a_loss: 0.4636 - d_loss: 0.4666 - v_ccc: 0.6753 - a_ccc: 0.5362 - d_ccc: 0.5333 - val_loss: 2.2075 - val_v_loss: 0.7644 - val_a_loss: 0.7079 - val_d_loss: 0.7442 - val_v_ccc: 0.2306 - val_a_ccc: 0.3002 - val_d_ccc: 0.2618
Epoch 23/50
2170/2170 [==============================] - 2s 1ms/step
[0.27419397234916687, 0.33518534898757935, 0.22023344039916992]
19
